<h1 style="margin:0; line-height:1.5; padding:0;">Intro to Computational Thinking and Programming - Group 4</h1>
<p style="margin:0;line-height:1; padding:0; font-size:20px;">Team Allocation Simulator</p>
<h5 style="margin:0; margin-top:10px; line-height:1; padding:0; padding-top:10px; color:rgba(231, 60, 60, 1); text-decoration: underline; font-weight:bold;">Objective:</h5>
<ol>
    <li>School Affliation: No majority of students from the same school</li>
    <li>Gender: No majority of students of the same gender</li>
    <li>Current CGPA: To balance academic performance</li>
</ol>

<p>This block imports the necessary libraries, provides the path to the data file
and defines a variable for the teamsize to allow for further customisation</p>

<style>
ol{
    line-height:.2;
    margin-top:5px;
    padding-top:5px;
}

li{
    line-height:1.3;
}
</style>

In [2]:
import csv
import matplotlib.pyplot as plt
import json
path = "records.csv" #this is the path to the data file
teamsize = 5

<h5 style="text-decoration:underline; margin:0; padding:0; line-height:1; font-weight:bold;">Reading and Sorting Data</h5>
<p>Define a function called <b>tut_grp_json</b> that reads data from the <b>records.csv file</b> and creates a <b>tut_grp.json file</b> that contains the students details sorted based on their tutorial group in a numerical order</p>
<p>Pseudocode:</p>
<pre>
Function tut_grp_json(path, json_filename='tut_grp'):
    1. Append '.json' to json_filename
    2. Create empty dictionary data to store results
    3. Open CSV file at path for reading:
        a. Create DictReader for the file
        b. For each row in the CSV:
            - Extract and remove 'Tutorial Group' from row as key
            - If key not in data:
                Create new list with row as first item
            - Else:
                Append row to existing list for that key
        
    4. Sort tutorial group keys numerically:
        a. Create empty key_list
        b. For each key in data:
            - Split key at '-' and convert number to integer
            - Add number to key_list
        c. Sort key_list numerically
        d. Convert numbers back to 'G-X' format
    
    5. Create new dictionary with sorted keys:
        - Use dictionary comprehension to create student_data
        - Keys are sorted 'G-X' strings
        - Values are lists of student data from original dict
    
    6. Write to JSON file:
        - Open json_filename for writing
        - Dump student_data with indent=2 for readability
    
    7. Return json_filename
</pre>

In [3]:
def tut_grp_json(path: str,json_filename: str='tut_grp'): #path: takes in filepath of csv file; json_filename: specify the file name of the json file without the filetype
    json_filename=json_filename+'.json'
    data={} #overall data
    with open(path,'r',newline='') as fObj:
        reader=csv.DictReader(fObj)
        for row in reader:
            key=row.pop('Tutorial Group') #remove the tutorial group from the row and store the variable
            if key not in data:
                data[key]=[row] #the value of data[key] is a list; if key not in, create new key
            else:
                data[key].append(row) #if key already exist, append to the existing list corresponding to the key
        ###sorting out the keys to make sure they are in order
        key_list=[]
        for key,student_info in data.items():
            key_list.append(int(key.split('-')[1])) #split string at '-'.
            key_list.sort() #sort from 1-120 in numerical order because python does not sort 'G-xx' in numerical order
        def myFunc(i):
            return 'G-'+str(i)
        key_list=list(map(myFunc,key_list))
        student_data = {key: data[key] for key in key_list}

        ###writing the the student_data object into a json file
        with open(json_filename,'w') as fObj:
            #json file cannot be modifiable
            json.dump(student_data,fObj,indent=2) #write to json file so the sorted dictionary cannot be accidentally messed up
        return json_filename


<h5 style="line-height:.5; margin:0; padding:0; font-weight:bold;">Why .json?</h5>
<p>As data stored in a .json file cannot be modified. Thus, ensuring that data is not modified while being used in the program.</p>
<h5 style="text-decoration:underline; line-height:1; margin:20px 0 0 0; padding:20px 0 0 0; font-weight:bold;">Loading Sorted Data into Python Program</h5>
<p>Define a function called <b>load_json_to_dict</b> that returns sorted data from the tut_grp.json file into a global variable called values that called the function to store data in a dictionary</p>

In [4]:
def load_json_to_dict(json_file: str): #loads json file to a dictionary and returns a dictionary
    with open(json_file,'r') as jFileObj:
        values=json.load(jFileObj)
    return values

In [5]:
json_file=tut_grp_json(path)

values=load_json_to_dict(json_file)